In [1]:
from datasets import load_dataset

ds = load_dataset("shunk031/jsnli", 'with-filtering')

0000.parquet:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

with-filtering/validation/0000.parquet:   0%|          | 0.00/301k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/533005 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3916 [00:00<?, ? examples/s]

In [2]:
rows = list(ds['train']) + list(ds['validation'])
print(len(rows))

# 2 contradiction
# 0 entailment
rows = [r for r in rows if r['label'] != 1]
print(len(rows))

536921
357597


In [3]:
sents = set([r['premise'] for r in rows] + [r['hypothesis'] for r in rows])
sent2id = {s:i for i, s in enumerate(sents)}
id2sent = {i:s for s, i in sent2id.items()}
pairs_with_lb = [(set([sent2id[r['premise']], sent2id[r['hypothesis']]]), r['label']) for r in rows]

In [4]:
sent_lb_count = {}
for r in rows:
    idx = sent2id[r['premise']]
    if idx not in sent_lb_count:
        sent_lb_count[idx] = set()
    sent_lb_count[idx].add(r['label'])

    idx = sent2id[r['hypothesis']]
    if idx not in sent_lb_count:
        sent_lb_count[idx] = set()
    sent_lb_count[idx].add(r['label'])

lb_sent_count = {}
for ls in sent_lb_count.values():
    if str(ls) not in lb_sent_count:
        lb_sent_count[str(ls)] = 0
    lb_sent_count[str(ls)] += 1

lb_sent_count

{'{2}': 150365, '{0, 2}': 148262, '{0}': 134629}

In [5]:
# from collections import defaultdict

# shared_index_pairs = defaultdict(list)

# for (idx_set, label) in pairs_with_lb:
#     if len(idx_set) == 1:
#         continue
#     idx1, idx2 = idx_set
#     shared_index_pairs[idx1].append((idx2, label))
#     shared_index_pairs[idx2].append((idx1, label))

# triplets = set()
# for idx, connections in shared_index_pairs.items():
#     # Separate connections by label
#     label_0_connections = {conn for conn, lbl in connections if lbl == 0}
#     label_2_connections = {conn for conn, lbl in connections if lbl == 2}
#     break

In [6]:
# shared_index_pairs
# label_0_connections
# label_2_connections

# item = pairs_with_lb[300000]
# print(id2sent[list(item[0])[0]])
# print(id2sent[list(item[0])[1]])
# print(item[1])

In [7]:
# shared_index_pairs

In [8]:
from collections import defaultdict

def create_triplets(pairs):
    shared_index_pairs = defaultdict(list)
    
    # Map each index to its corresponding pairs and labels
    for (idx_set, label) in pairs:
#         print(idx_set)
        if len(idx_set) == 1:
            continue
        idx1, idx2 = idx_set
        shared_index_pairs[idx1].append((idx2, label))
        shared_index_pairs[idx2].append((idx1, label))
    
    # Create triplets
    triplets = {}
    for idx, connections in shared_index_pairs.items():
        # Separate connections by label
        label_0_connections = {conn for conn, lbl in connections if lbl == 0}
        label_2_connections = {conn for conn, lbl in connections if lbl == 2}
        
        # Create all triplets with one 0-label connection and one 2-label connection
        for idx0 in label_0_connections:
            for idx2 in label_2_connections:
                # Form the triplet and add to the result set
                triplet = [idx, idx0, idx2]
                triplets[str(triplet)] = triplet
    
    # Return the list of unique triplets
    return list(triplets.values())

In [9]:
triplets = create_triplets(pairs_with_lb)

In [10]:
triplet_samples = [
    {
        'sent0': id2sent[t[0]],
        'sent1': id2sent[t[1]],
        'hard_neg': id2sent[t[2]],
    } for t in triplets
]

In [11]:
import pandas as pd

df = pd.DataFrame(triplet_samples)
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv("jsnli_for_simcse.csv", index=False)

In [12]:
df.duplicated(['sent0', 'sent1', 'hard_neg']).sum()

0

In [13]:
df.head()

,sent0,sent1,hard_neg
0,黒い 服 を 着た ハゲ 男 が 地下鉄 を 待って おり 、 柱 に 寄りかかって いる ...,ハゲ 男 は すべて 黒 を 着て い ます 。,ハゲ 男 は すべて 白 を 着て い ます 。
1,水着 の 男性 と 女性 は 、 木々 を 背景 に した 丸太 の 汗 の ロッジ の 外...,男性 と 女性 は 汗 の ロッジ の 外 に 座って い ます 。,男性 と 女性 は プール で 泳ぎ ます 。
2,男 は 裸 です,風光明媚な 山頂 で 、 ネット の 裸 の 男 が 眠り に つく 。,ステージ で 歌 と 観客 を 指す 黒い ジャケット と ズボン を 身 に 着けて いる...
3,女性 が 通り を 歩き ます 。,混雑 した 通り を 歩いて 、 髪 に 三 つ 編み の ピンク の 白 と オレンジ の...,オレンジ コーン を 飛び越えて 黄色 の シャツ を 着て いる 少女 。
4,４ 人 の 男性 が 椅子 に 座って おり 、 その 前 に 音楽 が あり 、 ４ 人 ...,４ 人 の 男性 が 座って い ます 。,４ 人 の 男性 が 音楽 を 演奏 し ながら 踊って い ます 。
